# Quantitative Risk Management

Click <a href="https://colab.research.google.com/github/Lolillosky/QuantRiskManagement/blob/main/NOTEBOOKS/3_AD_Pytorch.ipynb">
    <img src="https://upload.wikimedia.org/wikipedia/commons/d/d0/Google_Colaboratory_SVG_Logo.svg" width="30" alt="Google Colab">
</a> to open this notebook in Google Colab.

## Introduction to AD in Pytorch

In order to install Pythorch in your machine follow the instructions from [Pythorch help](https://pytorch.org/). The library is already installed in Google Colab environment.

In order to import the library:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

### Tensors

In order to be able to compute derivatives, we have to work with Pytorch tensors. These can be initialized from hardcoded values, numpy variables or Pythorch functions.

In [ ]:
x = torch.tensor(3.0)

y_numpy = np.linspace(0,2*np.pi,10)

y = torch.tensor(y_numpy)

z = torch.linspace(0,2*np.pi,10)

print(x)
print(y)
print(z)

The floating point precission can be set when a variable is created. 

In [ ]:
x = torch.tensor(3.0, dtype= torch.float64)

y_numpy = np.linspace(0,2*np.pi,10)

y = torch.tensor(y_numpy, dtype= torch.float64)

z = torch.linspace(0,2*np.pi,10, dtype= torch.float64)

print(x)
print(y)
print(z)

Default floating point precision can alse be set

In [ ]:
torch.set_default_dtype(torch.float64)

### Computing derivatives

In order to be able to perform AD, we must specify it.

In [ ]:
x = torch.tensor(3.0, requires_grad=True)
y = torch.tensor(2.0, requires_grad=True)

z = x**2 + x*y

z.backward()

print(x.grad)
print(y.grad)


With the exception of basic operations (+,-*,/), we must use pytorch functions. Pytorch functions and usage resemble numpy. 

In [ ]:
x = torch.linspace(0, 2*np.pi, 100, requires_grad=True)
y = torch.sin(x)
z = torch.sum(y)

z.backward()

grad = x.grad

plt.plot(x.detach().numpy(), y.detach().numpy(), label = 'sin(x)')
plt.plot(x.detach().numpy(), grad, label = r'$\frac{d\sin(x)}{dx}$')

plt.legend()



We can extract the numerical content of every tensor, but in AD has been enabled on the particular tensor, we must first detach it.

In [ ]:
x.numpy()

In [ ]:
x.detach().numpy()

Whenever we compute gradients, the tape is deleted unless we tell Pytorch not to.

In [ ]:
x = torch.tensor([1.0,2.0,3.0], requires_grad=True)

y = torch.tensor([1.5,3.25,3.47], requires_grad=True)

z1 = torch.sum(x**2 - y**2)
z2 = z1**2


z1.backward()
z2.backward()



In [ ]:
x = torch.tensor([1.0,2.0,3.0], requires_grad=True)

y = torch.tensor([1.5,3.25,3.47], requires_grad=True)

z1 = torch.sum(x**2 - y**2)
z2 = z1**2


z1.backward(retain_graph=True)
z2.backward()

print(x.grad)
print(y.grad)



We can disable tape recording without setting requires_grad to false:

In [ ]:
x = torch.linspace(0, 2*np.pi, 100, requires_grad=True)

y1 = torch.sin(x)

with torch.no_grad():
    y2 = torch.sin(x)

print(y1)
print(y2)


## Computing the Jacobian Matrix

Let us first define a function that takes several inputs and outputs. For example, a formula that computes the Montecarlo price of both a call and a put option given a set of parameters. 

In [ ]:
def MC_payoffs(spot, strike, vol, r, div, ttm, num_sims):

    brow = torch.tensor(np.random.normal(0,1,num_sims))*torch.sqrt(ttm)
    
    spot_mat = spot*torch.exp((r-div-0.5*vol*vol)*ttm + vol*brow)

    call = torch.mean(torch.maximum(spot_mat - strike, torch.tensor(0.0)))*torch.exp(-r*ttm)
    put = torch.mean(torch.maximum(-spot_mat + strike, torch.tensor(0.0)))*torch.exp(-r*ttm)

    return (call, put)
    

We make a wrapper to this function, so that the wrapper only takes as inputs the parameters with respect to which we want to compute the Jacobian.

In [ ]:

spot = torch.tensor(1.0, requires_grad=True)
strike = torch.tensor(1.0, requires_grad=True)
vol = torch.tensor(0.2, requires_grad=True)
r = torch.tensor(0.01, requires_grad=True)
div = torch.tensor(0.005, requires_grad=True)
ttm = torch.tensor(1.0, requires_grad=True)
num_sims = 50000

MC_100000 = lambda spot, strike, vol, r, div, ttm: MC_payoffs(spot, strike, vol, r, div, ttm, 100000)



To compute the Jacobian, do the following:

In [ ]:
from torch.autograd.functional import jacobian
jacobian(MC_100000, (spot, strike, vol, r, div, ttm))

## High order differentials

To compute high order differential, we must use torch.autograd.grad function iteratively

In [ ]:
x = torch.tensor(2.0, requires_grad=True)

y = 3*x**3-2.5*x**2+x+1

first_der = torch.autograd.grad(y,x,retain_graph= True, create_graph= True)

print(first_der)

second_der = torch.autograd.grad(first_der,x,retain_graph= True, create_graph= True)

print(second_der)
